In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10, cifar100
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()

# 5,000 random samples
sample_size = 5000
selected_indices = np.random.choice(x_train_full.shape[0], sample_size, replace=False)
x_train_sampled = x_train_full[selected_indices]
y_train_sampled = y_train_full[selected_indices]

# Normalize images
x_train_sampled = x_train_sampled.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to categorical format
y_train_sampled = to_categorical(y_train_sampled, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Split into train, validation, and test
x_train, x_temp, y_train, y_temp = train_test_split(x_train_sampled, y_train_sampled, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Print dataset shapes
print("Training set shape:", x_train.shape)
print("Validation set shape:", x_val.shape)
print("Test set shape:", x_test.shape)

In [ ]:
# Data augmentation setup
datagen = ImageDataGenerator(
    rotation_range=18,  # Slightly different rotation
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Apply to training data
datagen.fit(x_train)


In [ ]:
####MODEL####

In [ ]:
# Load CIFAR-100 dataset
(x_train_cifar100, y_train_cifar100), (x_test_cifar100, y_test_cifar100) = cifar100.load_data()

# Normalize images
x_train_cifar100 = x_train_cifar100.astype('float32') / 255.0
x_test_cifar100 = x_test_cifar100.astype('float32') / 255.0

# Convert labels to categorical format
y_train_cifar100 = to_categorical(y_train_cifar100, num_classes=100)
y_test_cifar100 = to_categorical(y_test_cifar100, num_classes=100)

# Split into train, validation, and test
x_train_cifar100, x_val_cifar100, y_train_cifar100, y_val_cifar100 = train_test_split(
    x_train_cifar100, y_train_cifar100, test_size=0.2, random_state=42)

# Data augmentation setup
datagen_cifar100 = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
datagen_cifar100.fit(x_train_cifar100)

In [ ]:
# Define the CNN model
cifar100_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001), input_shape=(32, 32, 3)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(100, activation='softmax', kernel_regularizer=l2(0.001))
])

In [ ]:
# Compile the model
cifar100_model.compile(optimizer=Adam(learning_rate=0.0005),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

# Display model summary
cifar100_model.summary()

# Define EarlyStopping and ReduceLROnPlateau callbacks
early_stop_cifar100 = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
lr_scheduler_cifar100 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

# Train the CIFAR-100 model
history_cifar100 = cifar100_model.fit(
    datagen_cifar100.flow(x_train_cifar100, y_train_cifar100, batch_size=32),
    epochs=50,
    validation_data=(x_val_cifar100, y_val_cifar100),
    callbacks=[early_stop_cifar100, lr_scheduler_cifar100],
    verbose=1
)

# Evaluate on CIFAR-100 test set
test_loss_cifar100, test_acc_cifar100 = cifar100_model.evaluate(x_test_cifar100, y_test_cifar100, verbose=1)
print(f"Test Loss (CIFAR-100): {test_loss_cifar100:.4f}")
print(f"Test Accuracy (CIFAR-100): {test_acc_cifar100:.4f}")

In [ ]:
from tensorflow.keras.regularizers import l2

# Modify model for CIFAR-10 fine-tuning
cifar100_model.pop()
cifar100_model.add(Dense(10, activation='softmax', kernel_regularizer=l2(0.002)))

# Unfreeze last few layers for fine-tuning
for layer in cifar100_model.layers[-10:]:  # Unfreezing last 10 layers
    layer.trainable = True

# Compile the fine-tuned model with a lower learning rate
cifar100_model.compile(optimizer=Adam(learning_rate=2e-5),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

# Train the fine-tuned model on CIFAR-10
tune_history = cifar100_model.fit(
    x_train, y_train,
    batch_size=8,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping, lr_scheduler],
    verbose=1
)

# Evaluate on CIFAR-10 test set
test_loss_tuned, test_acc_tuned = cifar100_model.evaluate(x_test, y_test, verbose=1)
print(f"Test Loss (Fine-tuned on CIFAR-10): {test_loss_tuned:.4f}")
print(f"Test Accuracy (Fine-tuned on CIFAR-10): {test_acc_tuned:.4f}")
